In [1]:
import abc
import six
import operator
import dask
import dask.array as da
import dask.base
import numpy as np

In [2]:
@six.add_metaclass(abc.ABCMeta)
class DaskLinearOperator(dask.base.DaskMethodsMixin):
    def __init__(self, dsk, keys, shape, chunks, dtype,
                 opname='linear-operator'):
        self._dsk = dsk
        self._keys = keys
        self._opname = str(opname) + '-'
        self.shape = shape
        self.chunks = chunks
        self.nblocks = tuple(len(cc) for cc in chunks)
        self.dtype = dtype

    def __dask_graph__(self):
        return self._dsk

    def __dask_keys__(self):
        return self._keys

    @staticmethod
    def __dask_optimize__(dsk, keys, **kwargs):
        return dsk

    # Use the threaded scheduler by default.
    __dask_scheduler__ = staticmethod(dask.threaded.get)

    def __dask_postcompute__(self):
        # We want to return the results as a tuple, so our finalize
        # function is `tuple`. There are no extra arguments, so we also
        # return an empty tuple.
        return dask.array.Array, ()

    def __dask_postpersist__(self):
        return DaskLinearOperator, (self._keys,)

    def __dask_tokenize__(self):
        return (self._opname,) + tuple(self._keys)

    @abc.abstractmethod
    def graph_apply(self, dsk, input_key, output_key, **options):
        pass


In [4]:
@six.add_metaclass(abc.ABCMeta)
class DLOSymmetric(DaskLinearOperator):
    def __init__(self, dsk, shape, chunks, dtype):
        assert shape[0] == shape[0]
        assert chunks[0] == chunks[1]
        DaskLinearOperator.__init__(self, dsk, shape, chunks, dtype, optype='symmetric-DLO')

In [5]:
class DLODense(DaskLinearOperator):
    def __init__(self, array, name=None):
        assert isinstance(array, da.Array), 'input is dask.array.Array'
        self.data = array
        DaskLinearOperator.__init__(self, 
                                    array.dask, array.shape, array.chunks, array.dtype, 
                                    optype='dense-DLO')

    def graph_apply(self, dsk, input_key, output_key, transpose=False,
                    **options):
        if not transpose:
            idx_out, idx_arr, idx_in = 'i', 'ij', 'j'
            blk_arr, blk_in = self.nblocks, (self.nblocks[0],)
            fcn = None
        else:
            idx_out, idx_arr, idx_in = 'j', 'ji', 'i'
            blk_arr, blk_in = self.nblocks[::-1], (self.nblocks[1],)
            fcn = da.transpose

        dsk_out = da.core.top(
                da.core.dotmany,
                output_key, idx_out,
                self.data.name, idx_arr,
                input_key, idx_in,
                leftfunction=fcn,
                numblocks={self.data.name: blk_arr, input_key: blk_in})
        return dask.sharedict.merge(self.dask, (dsk_out, output_key))

TypeError: Can't instantiate abstract class DLOSymmetric with abstract methods graph_apply